# NAS - Optuna

- **Authored by:** Matheus Ferreira Silva 
- **GitHub:**: https://github.com/MatheusFS-dev

## 1. Setup and Configuration

### 1.1. Environment Variables

In [ ]:
import os

# Async CUDA allocator
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# If cuDNN autotune fails, fall back to a safe (but slower) algorithm.
os.environ["XLA_FLAGS"] = "--xla_gpu_strict_conv_algorithm_picker=false"

# Allow TensorFlow to allocate GPU memory as needed
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 

### 1.2. Imports

In [ ]:
from _imports import * # Centralized file containing all imports

### 1.3. GPU Management

In [ ]:
# Specify GPU to use (e.g., GPU 0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
troo.get_gpu_info()

## 2. Run Parameters 

In [ ]:
TOTAL_NUM_PORTS = 100

observed_ports_list = [3, 4, 5, 6, 7, 10, 15]
batch_sizes = [64, 256, 64, 256, 256, 256, 128]

model_paths = [
    "./results/models/lstm/optuna_study_3_ports/models/trial_107.keras",
    "./results/models/lstm/optuna_study_4_ports/models/trial_200.keras",
    "./results/models/lstm/optuna_study_5_ports/models/trial_94.keras",
    "./results/models/lstm/optuna_study_6_ports/models/trial_121.keras",
    "./results/models/lstm/optuna_study_7_ports/models/trial_178.keras",
    "./results/models/lstm/optuna_study_10_ports/models/trial_199.keras",
    "./results/models/lstm/optuna_study_15_ports/models/trial_120.keras",
]

scalers = [
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
]

THRESHOLD = 0.95
SNR_LINEAR = 1.25

mixed_precision.set_global_policy("mixed_float16")

In [ ]:
RUN_DIR = troo.create_run_directory(prefix="lstm_op_")
print(f"Run directory: {RUN_DIR}")

## 3. Data Loading and Preprocessing

In [ ]:
# --------------------- Load the dataset in matlab format -------------------- #
rng = np.random.default_rng(42)

kappa0_mu1_m0 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu1.0_m0.0.mat")["SNR_events"]
kappa0_mu1_m2 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu1.0_m2.0.mat")["SNR_events"]
kappa0_mu1_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu1.0_m50.0.mat")["SNR_events"]
kappa0_mu2_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu2.0_m50.0.mat")["SNR_events"]
kappa0_mu5_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu5.0_m50.0.mat")["SNR_events"]
kappa5_mu1_m0 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu1.0_m0.0.mat")["SNR_events"]
kappa5_mu1_m2 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu1.0_m2.0.mat")["SNR_events"]
kappa5_mu1_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu1.0_m50.0.mat")["SNR_events"]
kappa5_mu2_m0 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu2.0_m0.0.mat")["SNR_events"]
kappa5_mu2_m2 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu2.0_m2.0.mat")["SNR_events"]
kappa5_mu2_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu2.0_m50.0.mat")["SNR_events"]
kappa5_mu5_m0 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu5.0_m0.0.mat")["SNR_events"]
kappa5_mu5_m2 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu5.0_m2.0.mat")["SNR_events"]
kappa5_mu5_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu5.0_m50.0.mat")["SNR_events"]

# ————————————— Split the data into 10% training and 90% testing ————————————— #

# kappa0_mu1_m0
perm = rng.permutation(kappa0_mu1_m0.shape[0])
n_test = int(0.9*kappa0_mu1_m0.shape[0])
kappa0_mu1_m0_test = kappa0_mu1_m0[perm[:n_test]]
kappa0_mu1_m0 = kappa0_mu1_m0[perm[n_test:]]

# kappa0_mu1_m2
perm = rng.permutation(kappa0_mu1_m2.shape[0])
n_test = int(0.9*kappa0_mu1_m2.shape[0])
kappa0_mu1_m2_test = kappa0_mu1_m2[perm[:n_test]]
kappa0_mu1_m2 = kappa0_mu1_m2[perm[n_test:]]

# kappa0_mu1_m50
perm = rng.permutation(kappa0_mu1_m50.shape[0])
n_test = int(0.9*kappa0_mu1_m50.shape[0])
kappa0_mu1_m50_test = kappa0_mu1_m50[perm[:n_test]]
kappa0_mu1_m50 = kappa0_mu1_m50[perm[n_test:]]

# kappa0_mu2_m50
perm = rng.permutation(kappa0_mu2_m50.shape[0])
n_test = int(0.9*kappa0_mu2_m50.shape[0])
kappa0_mu2_m50_test = kappa0_mu2_m50[perm[:n_test]]
kappa0_mu2_m50 = kappa0_mu2_m50[perm[n_test:]]

# kappa0_mu5_m50
perm = rng.permutation(kappa0_mu5_m50.shape[0])
n_test = int(0.9*kappa0_mu5_m50.shape[0])
kappa0_mu5_m50_test = kappa0_mu5_m50[perm[:n_test]]
kappa0_mu5_m50 = kappa0_mu5_m50[perm[n_test:]]

# kappa5_mu1_m0
perm = rng.permutation(kappa5_mu1_m0.shape[0])
n_test = int(0.9*kappa5_mu1_m0.shape[0])
kappa5_mu1_m0_test = kappa5_mu1_m0[perm[:n_test]]
kappa5_mu1_m0 = kappa5_mu1_m0[perm[n_test:]]

# kappa5_mu1_m2
perm = rng.permutation(kappa5_mu1_m2.shape[0])
n_test = int(0.9*kappa5_mu1_m2.shape[0])
kappa5_mu1_m2_test = kappa5_mu1_m2[perm[:n_test]]
kappa5_mu1_m2 = kappa5_mu1_m2[perm[n_test:]]

# kappa5_mu1_m50
perm = rng.permutation(kappa5_mu1_m50.shape[0])
n_test = int(0.9*kappa5_mu1_m50.shape[0])
kappa5_mu1_m50_test = kappa5_mu1_m50[perm[:n_test]]
kappa5_mu1_m50 = kappa5_mu1_m50[perm[n_test:]]

# kappa5_mu2_m0
perm = rng.permutation(kappa5_mu2_m0.shape[0])
n_test = int(0.9*kappa5_mu2_m0.shape[0])
kappa5_mu2_m0_test = kappa5_mu2_m0[perm[:n_test]]
kappa5_mu2_m0 = kappa5_mu2_m0[perm[n_test:]]

# kappa5_mu2_m2
perm = rng.permutation(kappa5_mu2_m2.shape[0])
n_test = int(0.9*kappa5_mu2_m2.shape[0])
kappa5_mu2_m2_test = kappa5_mu2_m2[perm[:n_test]]
kappa5_mu2_m2 = kappa5_mu2_m2[perm[n_test:]]

# kappa5_mu2_m50
perm = rng.permutation(kappa5_mu2_m50.shape[0])
n_test = int(0.9*kappa5_mu2_m50.shape[0])
kappa5_mu2_m50_test = kappa5_mu2_m50[perm[:n_test]]
kappa5_mu2_m50 = kappa5_mu2_m50[perm[n_test:]]

# kappa5_mu5_m0
perm = rng.permutation(kappa5_mu5_m0.shape[0])
n_test = int(0.9*kappa5_mu5_m0.shape[0])
kappa5_mu5_m0_test = kappa5_mu5_m0[perm[:n_test]]
kappa5_mu5_m0 = kappa5_mu5_m0[perm[n_test:]]

# kappa5_mu5_m2
perm = rng.permutation(kappa5_mu5_m2.shape[0])
n_test = int(0.9*kappa5_mu5_m2.shape[0])
kappa5_mu5_m2_test = kappa5_mu5_m2[perm[:n_test]]
kappa5_mu5_m2 = kappa5_mu5_m2[perm[n_test:]]

# kappa5_mu5_m50
perm = rng.permutation(kappa5_mu5_m50.shape[0])
n_test = int(0.9*kappa5_mu5_m50.shape[0])
kappa5_mu5_m50_test = kappa5_mu5_m50[perm[:n_test]]
kappa5_mu5_m50 = kappa5_mu5_m50[perm[n_test:]]

# ————————————— Concatenate all training subsamples along axis=0 ————————————— #
dataset = np.concatenate(
    [
        kappa0_mu1_m0,
        kappa0_mu1_m2,
        kappa0_mu1_m50,
        kappa0_mu2_m50,
        kappa0_mu5_m50,
        kappa5_mu1_m0,
        kappa5_mu1_m2,
        kappa5_mu1_m50,
        kappa5_mu2_m0,
        kappa5_mu2_m2,
        kappa5_mu2_m50,
        kappa5_mu5_m0,
        kappa5_mu5_m2,
        kappa5_mu5_m50,
    ],
    axis=0,
)

print(f"Original dataset shape: {dataset.shape}")

# Subsample data
# dataset = dataset[: int(0.01 * dataset.shape[0]), :]

print(f"Shape of the data after configuration: {dataset.shape}\n")

## 4. Getters

### 4.6. Implementation getters

In [ ]:
def get_observed_ports(sinr_data, num_observed_ports, total_ports):
    """
    Extracts SINR values for the specified number of observed ports.

    The function selects a subset of SINR data by identifying equally spaced ports based on the
    number of observed ports specified. It returns the SINR values for these observed ports and
    their corresponding indices.

    Args:
        sinr_data (numpy.ndarray): A 2D array where each row represents an observation and each column
                                   represents a port with its corresponding SINR values.
        num_observed_ports (int): The number of observed ports to select from the SINR data.
        total_ports (int): The total number of ports in the SINR data.

    Returns:
        observed_sinr (numpy.ndarray): A 2D array containing the SINR values for the observed ports.
        observed_indices (numpy.ndarray): A 1D array of the indices corresponding to the observed ports.
    """
    observed_indices = np.linspace(0, total_ports - 1, num_observed_ports, dtype=int)
    observed_sinr = sinr_data[:, observed_indices]

    return observed_sinr, observed_indices


def getOP(
    observed_indices: np.ndarray,
    predicted_values: np.ndarray,
    true_values: np.ndarray,
    threshold: float,
    snr_linear: float,
    total_ports: int,
) -> float:
    """Estimate the outage probability for regression models.

    This function compares the predicted and observed signal values at different
    channels (ports) and determines whether the chosen signal is above a given threshold.
    The outage probability is then computed as the proportion of times the signal falls
    below this threshold.

    Args:
        observed_indices (np.ndarray): Indices of the observed ports (channels).
        predicted_values (np.ndarray): Matrix of predicted values for each sample.
        true_values (np.ndarray): Ground-truth values for each port.
        threshold (float): Threshold value for determining outage.
        snr_linear (float): Signal-to-noise ratio in linear scale.

    Returns:
        float: Estimated outage probability.
    """

    # Initialize an array with negative infinity to store the observed values
    observed_values_matrix = np.full((true_values.shape[0], total_ports), -np.inf, dtype=np.float64)

    # Assign the true values of the observed ports (channels) to the matrix
    observed_values_matrix[:, observed_indices] = true_values[:, observed_indices]

    # Find the index of the highest predicted value for each sample
    best_predicted_indices = np.argmax(predicted_values, axis=1)

    # Initialize an array with negative infinity to store the predicted values
    predicted_values_matrix = np.full((true_values.shape[0], total_ports), -np.inf, dtype=np.float64)

    # Assign the true value corresponding to the predicted best port
    predicted_values_matrix[np.arange(len(best_predicted_indices)), best_predicted_indices] = true_values[
        np.arange(len(best_predicted_indices)), best_predicted_indices
    ]

    # Take the element-wise maximum between the observed and predicted value matrices
    best_value_matrix = np.maximum(observed_values_matrix, predicted_values_matrix)

    # print("Shape of Best Value Matrix:", best_value_matrix.shape)

    # Find the index of the best predicted or observed port (channel) for each sample
    best_predicted_or_observed_ports = np.argmax(best_value_matrix, axis=1)

    # print("Shape of Best Predicted/Observed Ports:", best_predicted_or_observed_ports.shape)
    # print("Number of Selected Ports:", len(best_predicted_or_observed_ports))

    # Retrieve the actual values corresponding to the best selected ports
    selected_values = best_value_matrix[np.arange(len(true_values)), best_predicted_or_observed_ports]

    # print("Shape of Selected Values:", selected_values.shape)

    # Determine which selected values are above the given threshold
    above_threshold = selected_values > (threshold / snr_linear)

    # print("Shape of Above Threshold Array:", above_threshold.shape)

    # Compute the outage probability: probability that the selected value is below the threshold
    outage_probability = 1.0 - (np.sum(above_threshold) / len(true_values))

    return outage_probability


def getObservedOP(
    observed_indices: np.ndarray, true_values: np.ndarray, threshold: float, snr_linear: float
) -> float:
    """
    Outage probability when you only observe a subset of ports.

    For each sample, picks the best SINR among observed ports,
    then computes OP = 1 - P(best_obs > threshold/snr_linear).
    """
    # extract only observed-port SINRs
    observed_sinr = true_values[:, observed_indices]
    # best per sample
    best_obs = np.max(observed_sinr, axis=1)
    # fraction above threshold
    p_above = np.mean(best_obs > (threshold / snr_linear))
    return 1.0 - p_above


def getIdealOP(true_values: np.ndarray, threshold: float, snr_linear: float) -> float:
    """
    Genie‐aided outage probability knowing all ports.

    For each sample, picks the best SINR across all ports,
    then computes OP = 1 - P(best_all > threshold/snr_linear).
    """
    best_all = np.max(true_values, axis=1)
    p_above = np.mean(best_all > (threshold / snr_linear))
    return 1.0 - p_above

## MAIN 

In [ ]:
datasets = [
    kappa0_mu1_m0_test,
    kappa0_mu1_m2_test,
    kappa0_mu1_m50_test,
    kappa0_mu2_m50_test,
    kappa0_mu5_m50_test,
    kappa5_mu1_m0_test,
    kappa5_mu1_m2_test,
    kappa5_mu1_m50_test,
    kappa5_mu2_m0_test,
    kappa5_mu2_m2_test,
    kappa5_mu2_m50_test,
    kappa5_mu5_m0_test,
    kappa5_mu5_m2_test,
    kappa5_mu5_m50_test,
]

dataset_names: list[str] = [
    "kappa0_mu1_m0_test",
    "kappa0_mu1_m2_test",
    "kappa0_mu1_m50_test",
    "kappa0_mu2_m50_test",
    "kappa0_mu5_m50_test",
    "kappa5_mu1_m0_test",
    "kappa5_mu1_m2_test",
    "kappa5_mu1_m50_test",
    "kappa5_mu2_m0_test",
    "kappa5_mu2_m2_test",
    "kappa5_mu2_m50_test",
    "kappa5_mu5_m0_test",
    "kappa5_mu5_m2_test",
    "kappa5_mu5_m50_test",
]

In [9]:
# Precompute ideal OP once per dataset
ideal_ops_global = [
    getIdealOP(data, THRESHOLD, SNR_LINEAR)
    for data in datasets
]

for n_ports, model_path, batch_size, scaler in zip(observed_ports_list, model_paths, batch_sizes, scalers):
    # create and/or clear subfolder
    sub_dir = os.path.join(RUN_DIR, f"op_{n_ports}_observed_ports")
    os.makedirs(sub_dir, exist_ok=True)

    # load the single model for this n_ports
    model = tf.keras.models.load_model(model_path)

    # fit scaler on training split observed at n_ports
    observed_ports, _ = get_observed_ports(dataset, n_ports, TOTAL_NUM_PORTS)
    X_train, X_val, y_train, y_val = train_test_split(
        observed_ports, dataset, test_size=0.2, random_state=0, shuffle=True
    )
    scaler.fit(X_train)

    test_losses = []
    ops = []
    obs_ops = []

    # evaluate on each test subset
    for data, name in zip(datasets, dataset_names):
        X_test, idxs = get_observed_ports(data, n_ports, TOTAL_NUM_PORTS)
        X_test = scaler.transform(X_test).reshape(X_test.shape[0], 1, X_test.shape[1])
        loss = model.evaluate(X_test, data, batch_size=batch_size, verbose=1)
        test_losses.append(loss)

        y_pred = model.predict(X_test, verbose=1)
        op = getOP(idxs, y_pred, data, THRESHOLD, SNR_LINEAR, TOTAL_NUM_PORTS)
        ops.append(op)

        # compute observed-only OP
        obs_op = getObservedOP(idxs, data, THRESHOLD, SNR_LINEAR)
        obs_ops.append(obs_op)

    # print a concise summary
    print(f"\n=== {n_ports} observed ports ===")
    for name, loss, op, obs_op, ideal_op in zip(
        dataset_names, test_losses, ops, obs_ops, ideal_ops_global
    ):
        print(f"{name}: Loss={loss:.6f}, OP={op:.6f}, ObsOP={obs_op:.6f}")

    # save results to file
    results = {f"{n}_loss": l for n, l in zip(dataset_names, test_losses)}
    results.update({f"{n}_op": o for n, o in zip(dataset_names, ops)})
    results.update({f"{n}_obsOP": o for n, o in zip(dataset_names, obs_ops)})

    out_file = os.path.join(sub_dir, f"results_{n_ports}_ports.txt")
    troo.save_trial_params_to_file(filepath=out_file, params={}, **results)

# 3. Global ideal-OP section (once per dataset)
print("\n=== Ideal Outage Probability (genie-aided) per dataset ===")
for name, ideal_op in zip(dataset_names, ideal_ops_global):
    print(f"{name}: IdealOP={ideal_op:.6f}")

# (Optional) save to a dedicated file
ideal_results = {f"{n}_idealOP": o for n, o in zip(dataset_names, ideal_ops_global)}
ideal_file = os.path.join(RUN_DIR, "ideal_ops_global.txt")
troo.save_trial_params_to_file(filepath=ideal_file, params={}, **ideal_results)

28125/28125 ━━━━━━━━━━━━━━━━━━━━ 19s 647us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 2s 652us/step - loss: 0.0674
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 710us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 727us/step - loss: 0.0676
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 19s 669us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 791us/step - loss: 0.0461
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 21s 745us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 758us/step - loss: 0.0332
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 708us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 724us/step - loss: 11.2353
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 709us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 784us/step - loss: 0.0526
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 21s 753us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 724us/step - loss: 0.0385
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 22s 782us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 824us/step - loss: 10.0971
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 21s 729us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 2s 659us/step - loss: 0.0462
28125/28125 ━━━━━━━━━━━━━━

/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 16 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 12 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 813us/step - loss: 1.5651e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 19s 669us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 724us/step - loss: 1.5536e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 18s 628us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 2s 615us/step - loss: 1.5290e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 16s 577us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 2s 615us/step - loss: 9.7442e-05
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 604us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 2s 512us/step - loss: 7.9985e-05
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 14s 484us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 751us/step - loss: 0.0543
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 721us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 2s 654us/step - loss: 1.1009e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 600us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 2s 648us/step - loss: 8.4931e-05
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 18s 624us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 725us/step - loss: 0.0290
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 19s 664us/step


/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 16 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 12 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


7032/7032 ━━━━━━━━━━━━━━━━━━━━ 7s 751us/step - loss: 1.9340e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 601us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 4s 588us/step - loss: 1.8969e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 593us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 4s 585us/step - loss: 1.8675e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 597us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 5s 677us/step - loss: 9.3888e-05
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 21s 728us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 5s 697us/step - loss: 7.1661e-05
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 704us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 5s 667us/step - loss: 0.0318
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 718us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 4s 617us/step - loss: 1.1151e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 605us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 4s 585us/step - loss: 7.7666e-05
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 21s 732us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 5s 702us/step - loss: 0.0168
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 712us/step
